In [21]:
import pandas as pd
from pulp import LpMinimize, LpProblem, LpVariable, lpSum, LpBinary
import collections
from itertools import product

# 1. load data file
file_path = "demo-V5.xlsx"
activities_df = pd.read_excel(file_path, sheet_name="ActivityInfo")
classroom_df = pd.read_excel(file_path, sheet_name="ClassWeekInfo")
student_courses_df = pd.read_excel(file_path, sheet_name="StudentInfo")

### Model 1 (Course Schduling)：

In [22]:
# Index set
courses = student_courses_df["Course_ID"]
courses = set([int(num) for row in courses for num in row[1:-1].split(', ')])

classes = student_courses_df["Class_ID"]
classes = set([int(row) for row in classes])

classrooms = classroom_df["Classroom_ID"]
classrooms = set(int(row) for row in classrooms)

weeks = classroom_df["Available_Weeks"]
weeks = set([int(num) for row in weeks for num in row.split(', ')])

time_slots = classroom_df["Time_Slot"]
time_slots = set([int(num) for row in time_slots for num in row[1: -1].split(', ')])

activities = activities_df["Activity_ID"]
activities = set([int(row) for row in activities])

activity_types = activities_df["Type"]
activity_types = set(activity_types)

In [23]:
# parameters
B = ["morning", "afternoon"]
Gsa = [(row["Course_ID"], row["Activity_ID"]) for _, row in activities_df.iterrows() if row["Requires_Separation"] == 1]
Ysai = [(row["Course_ID"], row["Activity_ID"], row["Type"]) for _, row in activities_df.iterrows()]
Csa = [(row["Course_ID"], row["Activity_ID"]) for _, row in activities_df.iterrows() if row["Requires_Computers"] == 1]
Tsa = [(row["Course_ID"], row["Activity_ID"]) for _, row in activities_df.iterrows() if row["Requires_Tables"] == 1]
Bsab = [(row["Course_ID"], row["Activity_ID"], row["Schedule"]) for _, row in activities_df.iterrows() if row["Schedule"] in B]

Gamma_sa = [(row["Course_ID"], row["Activity_ID"]) for _, row in activities_df.iterrows() if row["Is_Exam"] == 1]
# CAPci = {(row["Classroom_ID"], row["Type"]): row["Capacity"] for _, row in classrooms.iterrows()} # 不准确
Pc = [row["Classroom_ID"] for _, row in classroom_df.iterrows() if row["Has_Computers"] == 1]
Ltb = [(0, "morning"), (1, "morning"), (2, "afternoon"), (3, "afternoon")]
Ltb_mapping = {row[0]: row[1] for row in Ltb}
Omega_tt = {}
for t1 in range(len(time_slots)):
    for t2 in range(len(time_slots)):
        if Ltb_mapping.get(t1, 0) == Ltb_mapping.get(t2, 0):
            Omega_tt[(t1, t2)] = 1
max_courses_per_week = 5
Psak = {(row["Course_ID"], row["Activity_ID"], k): int(week) 
        for _, row in activities_df.iterrows() 
        for k, week in enumerate(row["Preferred_Weeks"].split(', '))}
Wk = {k: penalty for k, penalty in zip(range(3), [0, 0.1, 0.3])}

activity_type_mapping = {row["Activity_ID"]: row["Type"] for _, row in activities_df.iterrows()}

In [24]:
available_weeks = {(row["Course_ID"], row["Activity_ID"]): row["Weeks_Scheduled_Candidates"].split(', ') for _, row in activities_df.iterrows()}
available_weeks = {k: [int(vv) for vv in v] for k, v in available_weeks.items()}
sa = [(s, a) for s in courses for a in activities_df[activities_df["Course_ID"] == s]["Activity_ID"]]

# 3. define model
model = LpProblem(name="course_scheduling", sense=LpMinimize)

# 变量
y = {(w, s, a): LpVariable(f"y_{w}_{s}_{a}", cat=LpBinary) for s, a in sa for w in available_weeks[(s, a)]}
x = {(w, t, s, a): LpVariable(f"x_{w}_{t}_{s}_{a}", cat=LpBinary) 
     for s, a in sa for w in available_weeks[(s, a)] for t in time_slots}

model += lpSum(Wk[k] * y[Psak[s, a, k], s, a] for k in range(1, 3) for s, a in sa if Psak.get((s, a, k), -1) != -1), "Minimize_Preference_Penalty"

# 4. A.2 - A.11)
# A.2
for s, a in sa:
    model += lpSum(x[w, t, s, a] for t in time_slots for w in available_weeks[(s, a)]) >= 1, f"Ensure_One_Assignment_{s}_{a}"

# A.3
for s, a in sa:
    for b in B:
        for w in available_weeks[(s, a)]:
            model += lpSum(
                x[w, t, s, a] for t in time_slots if (t, b) in Ltb
            ) <= 1, f"Max_One_Per_Limited_Slot_{s}_{a}_{b}_{w}"

# A.4
for s, a in sa:
    for b in B:
        if (s, a, b) not in Bsab:
            for w in available_weeks[(s, a)]:
                model += lpSum(x[w, t, s, a] for t in time_slots if (t, b) in Ltb) == 0, f"Forbidden_Resource_{s}_{a}_{b}_{w}"

# A.5
for s, a in sa:
    for t in time_slots:
        for t_prime in time_slots:
            for w in available_weeks[(s, a)]:
                if Omega_tt.get((t, t_prime), 0) == 0 or (s, a) not in Gsa:
                    model += x[w, t, s, a] + x[w, t_prime, s, a] <= y[w, s, a], f"Time_Slot_Limit_{w}_{s}_{a}_{t}_{t_prime}"

# A.6
for s, a in sa:
    for w in available_weeks[(s, a)]:
        for b in B:
            for b_prime in B:
                if b != b_prime or ((s, a, b) in B and (s, a, b_prime) in B):
                    model += (
                        lpSum(x[w, t, s, a] for t in time_slots if (t, b) in Ltb) +
                        lpSum(x[w, t_prime, s, a] for t_prime in time_slots if (t_prime, b_prime) in Ltb)
                    ) >= 2 * y[w, s, a], f"Resource_Conflict_{w}_{s}_{a}_{b}_{b_prime}"

# A.7
for w in weeks:
    model += lpSum(
        y[w, s, a] for s, a in sa if (s, a) in Gamma_sa and w in available_weeks[(s, a)]
    ) <= max_courses_per_week, f"Max_Courses_Per_Student_{w}"

# A.9
for t in time_slots:
    for s, a in sa:
        for s_prime, a_prime in sa:
            if s != s_prime and a != a_prime:
                for w in available_weeks[(s, a)]:
                    if (s, a) in Gsa and (s_prime, a_prime) in Gsa and (s, a) in Tsa and (s_prime, a_prime) in Tsa and w in available_weeks[(s_prime, a_prime)]:
                        model += x[w, t, s, a] + x[w, t, s_prime, a_prime] <= 1, f"Group_Time_Conflict_{w}_{t}_{s}_{s_prime}_{a}_{a_prime}"

# 5. solve
model.solve()

w_schedule_results = [
    {"Week": w, "Course_ID": s, "Activity_ID": a, "Probability": y[w, s, a].value()}
    for w, s, a in y if y[w, s, a].value() >= 0.5
]

w_t_schedule_results = [
    {"Week": w, "Time_slot": t, "Course_ID": s, "Activity_ID": a, "Probability": x[w, t, s, a].value()}
    for w, t, s, a in x if x[w, t, s, a].value() >= 0.5
]

w_schedule_df = pd.DataFrame(w_schedule_results)

w_t_schedule_df = pd.DataFrame(w_t_schedule_results)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/anaconda3/envs/pp2/lib/python3.12/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/th/3vtt05957_l2_90vqr26g_dr0000gn/T/bbb40ef4de054ef3a7bc8aa75474f693-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/th/3vtt05957_l2_90vqr26g_dr0000gn/T/bbb40ef4de054ef3a7bc8aa75474f693-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 3560 COLUMNS
At line 15623 RHS
At line 19179 BOUNDS
At line 20000 ENDATA
Problem MODEL has 3555 rows, 820 columns and 10397 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Problem is infeasible - 0.00 seconds
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.01   (Wallclock seconds):       0.01



In [25]:
w_schedule_df

,Week,Course_ID,Activity_ID,Probability
0,7,0,10011,1.0
1,5,0,10012,0.5
2,9,1,20011,0.5
3,4,1,20012,0.5
4,6,3,40011,0.5
5,11,3,40013,0.5
6,4,4,50011,0.5
7,4,5,60011,0.5
8,7,5,60012,0.5
9,4,6,70011,0.5


In [26]:
w_t_schedule_df

,Week,Time_slot,Course_ID,Activity_ID,Probability
0,7,2,0,10011,1.0
1,5,2,0,10012,0.5
2,5,3,0,10012,0.5
3,11,0,2,30011,1.0
4,9,1,2,30012,1.0
5,4,2,4,50011,0.5
6,4,3,4,50011,0.5
7,4,3,4,50012,1.0
8,4,0,6,70011,0.5
9,4,1,6,70011,0.5


##### **Result Processing**  
1. **Classify activity candidate sets by (week, time slot)**: For example, week 1 time slot 1, candidate set: {"course1", "course3", ...}. (The problem assumes that each subject can only hold one activity per week.)  

2. **Remove online exam activities**  

3. **Backup the results**: CSV format as follows  
|week|time_slot|subject_candidates|  

4. **Run Model 2 once for each time slot in each week**  

In [27]:
def process_data(df):
    df = df[~df["Activity_ID"].map(activity_type_mapping).eq(2)]
    grouped = df.groupby(["Week", "Time_slot"])["Course_ID"].apply(lambda x: set(x)).reset_index()
    grouped.rename(columns={"Course_ID": "subject_candidates"}, inplace=True)
    grouped.to_csv("processed_schedule.csv", index=False)
    return grouped

subject_candidates = process_data(w_t_schedule_df)
subject_candidates

,Week,Time_slot,subject_candidates
0,4,0,{6}
1,4,1,{6}
2,4,2,{8}
3,4,3,"{8, 4}"
4,5,2,{0}
5,5,3,{0}
6,7,2,{0}
7,9,1,{2}
8,11,0,"{8, 2}"
9,11,1,{8}


### Model 2 (Classroom Assignment)：

In [28]:
# parameters
Ysi = {(row["Course_ID"], row["Type"]): 1 for _, row in activities_df.iterrows()}
Gs = {row["Course_ID"]: 1 for _, row in activities_df.iterrows() if row["Requires_Separation"] == 1}
Cs = {row["Course_ID"]: 1 for _, row in activities_df.iterrows() if row["Requires_Computers"] == 1}
Ts = {row["Course_ID"]: 1 for _, row in activities_df.iterrows() if row["Requires_Tables"] == 1}
Tc = {row["Classroom_ID"]: 1 for _, row in classroom_df.iterrows() if row["Has_Tables"] == 1}
Act = {(row["Classroom_ID"], int(w), int(t)): 1 for _, row in classroom_df.iterrows() for w in row["Available_Weeks"].split(', ') for t in row["Time_Slot"][1: -1].split(', ')} #
Ic = {row["Classroom_ID"]: 1 for _, row in classroom_df.iterrows() if row["Is_Isolated"] == 1}
Ns = {row["Course_ID"]: row["Num_Students"] for _, row in activities_df.iterrows()}
Msg = collections.defaultdict(int)
for course in courses:
    for _, row in student_courses_df.iterrows():
        if str(course) in row["Course_ID"]:
            Msg[(course, row["Class_ID"])] += 1

class_courses = collections.defaultdict(set)
for (course, class_id), _ in Msg.items():
    class_courses[course].add(class_id)

Capacity = {row["Classroom_ID"]: row["Capacity"] for _, row in classroom_df.iterrows()}
Occupancy_rate = {1: 0.9, 2: 0, 3: 1}
CAPci = {(c, i): int(Capacity[c] * Occupancy_rate[i]) for c, i in product(classrooms, activity_types)}

In [30]:
df = pd.read_csv("processed_schedule.csv")
count = 0
week = df.iloc[count]["Week"]
time_slot = df.iloc[count]["Time_slot"]
subjects = df["subject_candidates"].tolist()
subjects = [int(i) for i in subjects[count][1:-1].split(', ')]

In [31]:
# 2. define model
model = LpProblem(name="classroom_assignment", sense=LpMinimize)

z = {(c, s): LpVariable(f"z_{c}_{s}", cat=LpBinary) for c in classrooms for s in subjects}
w = {(c, s, g): LpVariable(f"w_{c}_{s}_{g}", cat=LpBinary) for c in classrooms for s in subjects for g in class_courses[s]}

model += lpSum(z[c, s] for c in classrooms for s in subjects), "Minimize_Classroom_Usage"

# A.13
for s in subjects:
    model += lpSum(z[c, s] * sum(CAPci.get((c, i), 0) * Ysi.get((s, i), 0) for i in activity_types) for c in classrooms) >= Ns[s], f"Capacity_Constraint_{s}"

# A.14
for s in subjects:
    if Ts.get(s, 0) == 1:
        for g in class_courses[s]:
            model += lpSum(w[c, s, g] * sum(CAPci.get((c, i), 0) * Ysi.get((s, i), 0) for i in activity_types) for c in classrooms) >= Msg.get((s, g), 0), f"Classroom_Capacity_{s}_{g}"

# A.15
for c in classrooms:
    model += lpSum(z[c, s] for s in subjects) <= 1, f"Single_Assignment_{c}"

# A.16
for c in classrooms:
    for s in subjects:
        model += z[c, s] <= Act.get((c, week, time_slot), 0), f"Classroom_Availability_{c}_{s}"

# A.17
for c in classrooms:
    for s in subjects:
        if Gs.get(s, 0) == 1:
            for g in class_courses[s]:
                model += z[c, s] >= w[c, s, g], f"Consistency_1_{c}_{s}_{g}"

# A.18
for c in classrooms:
    for s in subjects:
        if Gs.get(s, 0) == 1:
            model += z[c, s] <= lpSum(w[c, s, g] for g in class_courses[s]), f"Consistency_2_{c}_{s}"

# A.19
for c in classrooms:
    for s in subjects:
        if Gs.get(s, 0) == 1:
            for g1 in class_courses[s]:
                for g2 in class_courses[s]:
                    if g1 != g2:
                        model += w[c, s, g1] + w[c, s, g2] <= 1, f"No_Shared_Classroom_{c}_{s}_{g1}_{g2}"

# A.20
for c in classrooms:
    for s in subjects:
        if Cs.get(s, 0) == 1 and c not in Pc:
            model += z[c, s] == 0, f"Computer_Requirement_{c}_{s}"

# A.21
for c in classrooms:
    for s in subjects:
        if Ts.get(s, 0) == 1 and Tc.get(c, 0) == 0:
            model += z[c, s] == 0, f"Table_Requirement_{c}_{s}"


# A.22
for c in classrooms:
    for s in subjects:
        if Ic.get(c, 0) == 1:
            model += lpSum(z[c_prime, s] for c_prime in classrooms if c_prime != c) <= (1 - z[c, s]) * len(classrooms), f"Isolation_Requirement_{c}_{s}"

# 3. solve
model.solve()

classroom_results = [
    {"Classroom": c, "Course_ID": s}
    for c, s in z if z[c, s].value() >= 0.5
]

classroom_assignment_df = pd.DataFrame(classroom_results)
classroom_assignment_df

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/anaconda3/envs/pp2/lib/python3.12/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/th/3vtt05957_l2_90vqr26g_dr0000gn/T/dc1213fee2cf454695786d38ab6a3b48-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/th/3vtt05957_l2_90vqr26g_dr0000gn/T/dc1213fee2cf454695786d38ab6a3b48-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 272 COLUMNS
At line 1283 RHS
At line 1551 BOUNDS
At line 1632 ENDATA
Problem MODEL has 267 rows, 80 columns and 830 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 0.2 - 0.00 seconds
Cgl0002I 14 variables fixed
Cgl0003I 12 fixed, 0 tightened bounds, 24 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 9 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 6 strengthened rows, 0 substitutions
Cg

,Classroom,Course_ID
0,107,6
1,109,6
2,116,6


In [32]:
results = []
for count in range(len(subject_candidates)):
    week = df.iloc[count]["Week"]
    time_slot = df.iloc[count]["Time_slot"]
    subjects = df["subject_candidates"].tolist()
    subjects = [int(i) for i in subjects[count][1:-1].split(', ')]

    # 2. define model
    model = LpProblem(name="classroom_assignment", sense=LpMinimize)

    z = {(c, s): LpVariable(f"z_{c}_{s}", cat=LpBinary) for c in classrooms for s in subjects}
    w = {(c, s, g): LpVariable(f"w_{c}_{s}_{g}", cat=LpBinary) for c in classrooms for s in subjects for g in class_courses[s]}

    model += lpSum(z[c, s] for c in classrooms for s in subjects), "Minimize_Classroom_Usage"

    # A.13
    for s in subjects:
        model += lpSum(z[c, s] * sum(CAPci.get((c, i), 0) * Ysi.get((s, i), 0) for i in activity_types) for c in classrooms) >= Ns[s], f"Capacity_Constraint_{s}"

    # A.14
    for s in subjects:
        if Ts.get(s, 0) == 1:
            for g in class_courses[s]:
                model += lpSum(w[c, s, g] * sum(CAPci.get((c, i), 0) * Ysi.get((s, i), 0) for i in activity_types) for c in classrooms) >= Msg.get((s, g), 0), f"Classroom_Capacity_{s}_{g}"

    # A.15
    for c in classrooms:
        model += lpSum(z[c, s] for s in subjects) <= 1, f"Single_Assignment_{c}"

    # A.16
    for c in classrooms:
        for s in subjects:
            model += z[c, s] <= Act.get((c, week, time_slot), 0), f"Classroom_Availability_{c}_{s}"

    # A.17
    for c in classrooms:
        for s in subjects:
            if Gs.get(s, 0) == 1:
                for g in class_courses[s]:
                    model += z[c, s] >= w[c, s, g], f"Consistency_1_{c}_{s}_{g}"

    # A.18
    for c in classrooms:
        for s in subjects:
            if Gs.get(s, 0) == 1:
                model += z[c, s] <= lpSum(w[c, s, g] for g in class_courses[s]), f"Consistency_2_{c}_{s}"

    # A.19
    for c in classrooms:
        for s in subjects:
            if Gs.get(s, 0) == 1:
                for g1 in class_courses[s]:
                    for g2 in class_courses[s]:
                        if g1 != g2:
                            model += w[c, s, g1] + w[c, s, g2] <= 1, f"No_Shared_Classroom_{c}_{s}_{g1}_{g2}"

    # A.20
    for c in classrooms:
        for s in subjects:
            if Cs.get(s, 0) == 1 and c not in Pc:
                model += z[c, s] == 0, f"Computer_Requirement_{c}_{s}"

    # A.21
    for c in classrooms:
        for s in subjects:
            if Ts.get(s, 0) == 1 and Tc.get(c, 0) == 0:
                model += z[c, s] == 0, f"Table_Requirement_{c}_{s}"


    # A.22
    for c in classrooms:
        for s in subjects:
            if Ic.get(c, 0) == 1:
                model += lpSum(z[c_prime, s] for c_prime in classrooms if c_prime != c) <= (1 - z[c, s]) * len(classrooms), f"Isolation_Requirement_{c}_{s}"

    # 3. solve
    model.solve()

    classroom_results = [
        {"Classroom": c, "Course_ID": s}
        for c, s in z if z[c, s].value() >= 0.5
    ]

    classroom_assignment_df = pd.DataFrame(classroom_results)
    results.append(classroom_assignment_df)
    classroom_assignment_df

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/anaconda3/envs/pp2/lib/python3.12/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/th/3vtt05957_l2_90vqr26g_dr0000gn/T/5f91cedd37664e24a35a0523b6c0b68b-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/th/3vtt05957_l2_90vqr26g_dr0000gn/T/5f91cedd37664e24a35a0523b6c0b68b-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 272 COLUMNS
At line 1283 RHS
At line 1551 BOUNDS
At line 1632 ENDATA
Problem MODEL has 267 rows, 80 columns and 830 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 0.2 - 0.00 seconds
Cgl0002I 14 variables fixed
Cgl0003I 12 fixed, 0 tightened bounds, 24 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 9 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 6 strengthened rows, 0 substitutions
Cg

In [33]:
results

[   Classroom  Course_ID
 0        107          6
 1        109          6
 2        116          6,
    Classroom  Course_ID
 0        107          6
 1        109          6
 2        116          6,
 Empty DataFrame
 Columns: []
 Index: [],
 Empty DataFrame
 Columns: []
 Index: [],
 Empty DataFrame
 Columns: []
 Index: [],
 Empty DataFrame
 Columns: []
 Index: [],
 Empty DataFrame
 Columns: []
 Index: [],
    Classroom  Course_ID
 0        110          2,
    Classroom  Course_ID
 0        107          8
 1        109          8
 2        110          2
 3        114          8
 4        119          8,
    Classroom  Course_ID
 0        107          8
 1        109          8
 2        114          8
 3        119          8]